## Embeddings

ယခင်ဥပမာတွင် ကျွန်ုပ်တို့သည် `vocab_size` အရှည်ရှိ အမြင့်မားသော dimension bag-of-words vectors များကို အသုံးပြုခဲ့ပြီး၊ အနိမ့် dimension positional representation vectors များမှ sparse one-hot representation သို့ တိတိကျကျ ပြောင်းလဲခဲ့ပါသည်။ သို့သော်၊ one-hot representation သည် memory ကို ထိရောက်စွာ အသုံးမပြုနိုင်သလို၊ စကားလုံးတစ်လုံးချင်းစီကိုလည်း အချင်းချင်း သီးခြားစီ ဆက်စပ်မှုမရှိဘဲ ဆက်ဆံထားသည်။ အဆိုပါ one-hot encoded vectors များသည် စကားလုံးများအကြား အဓိပ္ပါယ်ဆိုင်ရာ ဆင်တူမှုများကို မဖော်ပြနိုင်ပါ။

ဤယူနစ်တွင် ကျွန်ုပ်တို့သည် **News AG** dataset ကို ဆက်လက်လေ့လာသွားမည်ဖြစ်သည်။ စတင်ရန်အတွက် ဒေတာများကို load ပြုလုပ်ပြီး ယခင် notebook မှ အဓိပ္ပါယ်ဖွင့်ဆိုချက်များကို ရယူကြပါစို့။


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Embedding ဆိုတာဘာလဲ?

**Embedding** ဆိုတာက စကားလုံးတွေကို အနည်းဆုံး dimension ရှိတဲ့ dense vector တွေဖြင့် ကိုယ်စားပြုဖော်ပြတာဖြစ်ပြီး၊ စကားလုံးတစ်လုံးရဲ့ အဓိပ္ပါယ်ဆိုင်ရာ အနက်ကို တစ်နည်းနည်းနဲ့ ပြသပေးတာဖြစ်ပါတယ်။ စကားလုံး embedding တွေကို ဘယ်လို အဓိပ္ပါယ်ရှိအောင် ဖန်တီးမလဲဆိုတာကို နောက်ပိုင်းမှာ ဆွေးနွေးပေးပါမယ်၊ ဒါပေမယ့် အခုအချိန်မှာတော့ embedding ကို စကားလုံး vector ရဲ့ dimension ကို လျှော့ချတဲ့ နည်းလမ်းတစ်ခုအနေနဲ့ သာ စဉ်းစားကြည့်ပါ။

ဒါကြောင့် embedding layer က စကားလုံးတစ်လုံးကို input အနေနဲ့ ယူပြီး၊ သတ်မှတ်ထားတဲ့ `embedding_size` ရဲ့ output vector ကို ထုတ်ပေးပါမယ်။ တစ်နည်းအားဖြင့်၊ ဒါဟာ `Linear` layer နဲ့ ဆင်တူပါတယ်၊ ဒါပေမယ့် one-hot encoded vector ကို ယူမယ့်အစား၊ စကားလုံးနံပါတ်ကို input အနေနဲ့ ယူနိုင်မှာ ဖြစ်ပါတယ်။

ကျွန်တော်တို့ network ရဲ့ ပထမဆုံး layer အနေနဲ့ embedding layer ကို သုံးခြင်းအားဖြင့်၊ bag-of-words မှ **embedding bag** မော်ဒယ်ဆီကို ပြောင်းနိုင်ပါတယ်။ ဒီမှာ ကျွန်တော်တို့ရဲ့ စာသားထဲက စကားလုံးတစ်လုံးစီကို သက်ဆိုင်ရာ embedding ကို ပြောင်းပြီး၊ ထို embedding တွေကို `sum`၊ `average` သို့မဟုတ် `max` ကဲ့သို့သော aggregate function တစ်ခုခုကို တွက်ချက်ပေးပါမယ်။

![Image showing an embedding classifier for five sequence words.](../../../../../translated_images/my/embedding-classifier-example.b77f021a7ee67eee.webp)

ကျွန်တော်တို့ရဲ့ classifier neural network က embedding layer နဲ့ စပြီး၊ aggregation layer နဲ့ linear classifier ကို အပေါ်မှာ ထည့်သွင်းထားပါမယ်။


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### အရွယ်အစား မတူညီသော variable sequence ကို ကိုင်တွယ်ခြင်း

ဒီ architecture ရဲ့ ရလဒ်အနေနဲ့, network အတွက် minibatches တွေကို တစ်သတ်မှတ်နည်းလမ်းနဲ့ ဖန်တီးဖို့ လိုအပ်လာမယ်။ ယခင်ယူနစ်မှာ bag-of-words ကို အသုံးပြုတဲ့အခါ, minibatch ထဲမှာရှိတဲ့ BoW tensors အားလုံးဟာ `vocab_size` အရွယ်အစားတူညီနေခဲ့တယ်၊ ကျွန်တော်တို့ရဲ့ text sequence ရဲ့ အရှည်ကိုမထည့်သွင်းစဉ်းစားဘဲ။ ဒါပေမယ့် word embeddings ကို အသုံးပြုတဲ့အခါ, တစ်ခုချင်းစီ text sample မှာ စကားလုံးအရေအတွက် မတူညီလာနိုင်ပြီး, အဲ့ဒီ samples တွေကို minibatches အဖြစ် ပေါင်းစည်းတဲ့အခါ padding တစ်ခုခုကို အသုံးပြုဖို့ လိုအပ်လာမယ်။

ဒါကို datasource ကို `collate_fn` function ပေးခြင်းနဲ့ တူညီတဲ့နည်းလမ်းကို အသုံးပြုပြီး လုပ်ဆောင်နိုင်ပါတယ်။


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### အထွေထွေ Embedding Classifier ကိုလေ့ကျင့်ခြင်း

အခုတော့ သင့်တော်သော dataloader ကိုသတ်မှတ်ပြီးသားဖြစ်သောကြောင့် ယခင်ယူနစ်တွင်သတ်မှတ်ထားသောလေ့ကျင့်မှု function ကိုအသုံးပြု၍ မော်ဒယ်ကိုလေ့ကျင့်နိုင်ပါပြီ။


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **မှတ်ချက်**: အချိန်သက်သာစေရန်အတွက် ဒီမှာ ၂၅,၀၀၀ မှတ်တမ်းပမာဏ (တစ်ခုလုံးသော epoch ထက်နည်း) ကိုသာလေ့ကျင့်နေပါသည်၊ သို့သော် သင်သည် လေ့ကျင့်မှုကို ဆက်လက်လုပ်ဆောင်နိုင်ပြီး၊ အတော်များများသော epoch များအတွက် လေ့ကျင့်ရန် function တစ်ခုရေးနိုင်ပြီး၊ သင့်တိုးတက်မှုနှုန်း parameter ကို စမ်းသပ်ခြင်းဖြင့် ပိုမိုမြင့်မားသော တိကျမှုရရှိရန် ကြိုးစားနိုင်ပါသည်။ သင်သည် ၉၀% အနီးအတောင်တိကျမှုကို ရောက်ရှိနိုင်သင့်သည်။


### EmbeddingBag Layer နှင့် အရှည်မတူညီသော အစီအစဉ်ကို ကိုယ်စားပြုခြင်း

ယခင် အဆောက်အအုံတွင်၊ minibatch ထဲသို့ ထည့်သွင်းနိုင်ရန် အစီအစဉ်အားလုံးကို အရှည်တူအောင် pad လုပ်ရန် လိုအပ်ခဲ့သည်။ သို့သော်၊ အရှည်မတူညီသော အစီအစဉ်များကို ကိုယ်စားပြုရန်အတွက် ဤနည်းလမ်းသည် အကျိုးရှိဆုံးမဟုတ်ပါ။ အခြားနည်းလမ်းတစ်ခုမှာ **offset** vector ကို အသုံးပြုခြင်းဖြစ်ပြီး၊ ၎င်းသည် အစီအစဉ်အားလုံး၏ offsets ကို တစ်ခုတည်းသော vector အကြီးထဲတွင် သိမ်းဆည်းထားမည်ဖြစ်သည်။

![Offset sequence ကို ကိုယ်စားပြုထားသော ပုံ](../../../../../translated_images/my/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: အထက်ပါ ပုံတွင် အက္ခရာများ၏ အစီအစဉ်ကို ပြထားသော်လည်း၊ ကျွန်ုပ်တို့၏ ဥပမာတွင် စကားလုံးများ၏ အစီအစဉ်များနှင့် အလုပ်လုပ်နေပါသည်။ သို့သော်၊ offset vector ဖြင့် အစီအစဉ်များကို ကိုယ်စားပြုခြင်း၏ အခြေခံသဘောတရားမှာ မပြောင်းလဲပါ။

Offset ကိုယ်စားပြုမှုနှင့် အလုပ်လုပ်ရန်၊ [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) layer ကို အသုံးပြုပါသည်။ ၎င်းသည် `Embedding` နှင့် ဆင်တူသော်လည်း၊ content vector နှင့် offset vector ကို input အဖြစ် လက်ခံပြီး၊ `mean`, `sum` သို့မဟုတ် `max` ဖြစ်နိုင်သော averaging layer ကိုလည်း ပါဝင်စေသည်။

ဤနေရာတွင် `EmbeddingBag` ကို အသုံးပြုထားသော ပြင်ဆင်ထားသော network ကို ဖော်ပြထားသည်-


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

ဒေတာစနစ်ကိုလေ့ကျင့်ရန်အတွက် ပြင်ဆင်ရန် offset vector ကိုပြင်ဆင်ပေးမည့် conversion function ကိုပေးရန်လိုအပ်သည်။


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

ယခင် ဥပမာများနှင့် မတူကွဲပြားစွာ၊ ယခုအခါ ကျွန်ုပ်တို့၏ network သည် parameter နှစ်ခုကို လက်ခံပါသည် - data vector နှင့် offset vector၊ အရွယ်အစားကွဲပြားမှုရှိသည်။ ထို့အပြင် ကျွန်ုပ်တို့၏ data loader သည် 2 ခုအစား 3 ခုသောတန်ဖိုးများကိုပေးသည် - text နှင့် offset vector နှစ်ခုစလုံးကို feature အဖြစ်ပေးသည်။ ထို့ကြောင့် ကျွန်ုပ်တို့၏ training function ကို အနည်းငယ်ပြင်ဆင်ရန်လိုအပ်သည်။


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantic Embeddings: Word2Vec

ယခင်ဥပမာတွင် မော်ဒယ်၏ embedding layer သည် စကားလုံးများကို ဗက်တာအနေနဲ့ ဖော်ပြဖို့ သင်ယူခဲ့ပေမယ့်၊ အဲဒီဖော်ပြမှုမှာ အဓိပ္ပါယ်ဆိုင်ရာ အနက်အဓိပ္ပါယ်များ မပါရှိသလောက်ပါ။ စကားလုံးများကို ဗက်တာအနေနဲ့ ဖော်ပြရာမှာ၊ အဓိပ္ပါယ်တူသော စကားလုံးများ (သို့) အနီးစပ်ဆုံးသော စကားလုံးများသည် ဗက်တာအကွာအဝေး (ဥပမာ euclidian distance) အရ နီးစပ်နေမယ့် representation ကို သင်ယူနိုင်ရင် ပိုကောင်းမယ်လို့ ထင်ရပါတယ်။

ဒါကို ပြုလုပ်ဖို့အတွက်၊ ကျွန်တော်တို့ရဲ့ embedding မော်ဒယ်ကို စာသားအစုကြီးတစ်ခုအပေါ်မှာ အထူးနည်းလမ်းတစ်ခုနဲ့ ကြိုတင်သင်ယူစေဖို့ လိုအပ်ပါတယ်။ အဓိပ္ပါယ်ဆိုင်ရာ embedding များကို သင်ယူစေဖို့ ပထမဆုံးနည်းလမ်းများထဲက တစ်ခုကို [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) လို့ ခေါ်ပါတယ်။ ဒါဟာ စကားလုံးများကို ဖြန့်ဖြူးထားသော representation အဖြစ် ဖန်တီးဖို့ အသုံးပြုတဲ့ အဓိက architecture နှစ်ခုအပေါ်မှာ အခြေခံထားပါတယ်-

 - **Continuous bag-of-words** (CBoW) — ဒီ architecture မှာ မော်ဒယ်ကို ပတ်ဝန်းကျင်အကြောင်းအရာကနေ စကားလုံးတစ်လုံးကို ခန့်မှန်းဖို့ သင်ကြားပေးပါတယ်။ ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ ကို ပေးထားတဲ့အခါ၊ မော်ဒယ်ရဲ့ ရည်မှန်းချက်က $(W_{-2},W_{-1},W_1,W_2)$ ကနေ $W_0$ ကို ခန့်မှန်းဖို့ ဖြစ်ပါတယ်။
 - **Continuous skip-gram** — CBoW ရဲ့ ဆန့်ကျင်ဘက်ဖြစ်ပါတယ်။ ဒီမော်ဒယ်က ပတ်ဝန်းကျင်အကြောင်းအရာ စကားလုံးများကို အသုံးပြုပြီး လက်ရှိစကားလုံးကို ခန့်မှန်းပါတယ်။

CBoW က ပိုမြန်ပါတယ်၊ skip-gram က ပိုနှေးပေမယ့် မကြာခဏ မတွေ့ရတဲ့ စကားလုံးများကို ပိုကောင်းစွာ ဖော်ပြနိုင်ပါတယ်။

![CBoW နဲ့ Skip-Gram algorithm နှစ်ခုစလုံးကို စကားလုံးများကို ဗက်တာအဖြစ် ပြောင်းလဲဖို့ အသုံးပြုနေတဲ့ ပုံ။](../../../../../translated_images/my/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News dataset ပေါ်မှာ ကြိုတင်သင်ယူထားတဲ့ word2vec embedding ကို စမ်းသပ်ဖို့၊ **gensim** library ကို အသုံးပြုနိုင်ပါတယ်။ အောက်မှာ 'neural' နဲ့ အနီးဆုံးသော စကားလုံးများကို ရှာဖွေထားပါတယ်-

> **Note:** စကားလုံးဗက်တာများကို ပထမဆုံး ဖန်တီးတဲ့အခါ၊ အဲဒီဗက်တာများကို ဒေါင်းလုပ်လုပ်ရတာ အချိန်အတော်ကြာနိုင်ပါတယ်!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ကျွန်ုပ်တို့သည် စကားလုံးမှ ဗက်တာအင်ဘက်ဒ်များကိုလည်းတွက်ချက်နိုင်ပြီး အမျိုးအစားခွဲခြားမှုမော်ဒယ်ကိုလေ့ကျင့်ရန်အသုံးပြုနိုင်ပါသည် (ရှင်းလင်းမှုအတွက် ဗက်တာ၏ပထမ ၂၀ ခုအစိတ်အပိုင်းများကိုသာပြသပါသည်)။


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW နဲ့ Skip-Grams နှစ်ခုလုံးဟာ “predictive” embeddings ဖြစ်ပြီး၊ ဒါတွေက အနီးအနားက context တွေကိုပဲ အဓိကထားပါတယ်။ Word2Vec က global context ကို အသုံးမပြုပါဘူး။

**FastText** က Word2Vec အပေါ်မှာ တိုးတက်မှုတစ်ခုဖြစ်ပြီး၊ စကားလုံးတစ်လုံးစီအတွက် vector ကို သင်ယူရုံမက၊ စကားလုံးတစ်လုံးအတွင်းရှိ character n-grams တွေကိုပါ သင်ယူပါတယ်။ ဒီ representation တွေကို training တစ်ကြိမ်စီမှာ vector တစ်ခုအဖြစ် ပျမ်းမျှတွက်ချက်ပေးပါတယ်။ ဒါက pre-training အတွက် အချိန်ပိုကုန်စေသော်လည်း၊ word embeddings တွေမှာ sub-word information ကို encode လုပ်နိုင်စေပါတယ်။

အခြားနည်းလမ်းတစ်ခုဖြစ်တဲ့ **GloVe** က co-occurrence matrix ရဲ့ အယူအဆကို အသုံးပြုပြီး၊ co-occurrence matrix ကို neural methods တွေသုံးပြီး ပိုမိုဖော်ပြနိုင်တဲ့၊ nonlinear word vectors တွေအဖြစ် ခွဲထုတ်ပါတယ်။

သင် gensim က support လုပ်ပေးတဲ့ word embedding models အမျိုးမျိုးကို အသုံးပြုပြီး၊ embeddings ကို FastText နဲ့ GloVe အဖြစ် ပြောင်းလဲပြီး စမ်းသပ်ကြည့်နိုင်ပါတယ်။


## PyTorch တွင် Pre-Trained Embeddings ကို အသုံးပြုခြင်း

အထက်ပါ ဥပမာကို ပြောင်းလဲပြီး၊ Word2Vec ကဲ့သို့သော အဓိပ္ပါယ်ဆိုင်ရာ embeddings များဖြင့် ကျွန်ုပ်တို့၏ embedding layer တွင် matrix ကို ကြိုတင်ဖြည့်စွက်နိုင်ပါသည်။ Pre-trained embedding ၏ vocabulary နှင့် ကျွန်ုပ်တို့၏ text corpus ၏ vocabulary များသည် မျှမတူနိုင်ကြောင်းကို သတိပြုရမည်ဖြစ်ပြီး၊ မပါဝင်သော စကားလုံးများအတွက် weight များကို အလွတ်တစ်ဆင့်ဖြင့် initialize လုပ်မည်ဖြစ်သည်။


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

ကျွန်တော်တို့အနေဖြင့် တိကျမှုမှာ အလွန်များပြားသော တိုးတက်မှုကို မတွေ့ရပါဘူး၊ ဒါဟာ အလွန်ကွဲပြားတဲ့ ဝေါဟာရများကြောင့် ဖြစ်နိုင်ပါတယ်။  
ကွဲပြားတဲ့ ဝေါဟာရများကြောင့် ဖြစ်ပေါ်လာတဲ့ ပြဿနာကို ဖြေရှင်းရန် အောက်ပါနည်းလမ်းများကို အသုံးပြုနိုင်ပါတယ်-  
* word2vec မော်ဒယ်ကို ကျွန်တော်တို့ရဲ့ ဝေါဟာရပေါ်မှာ ပြန်လည်လေ့ကျင့်ပါ  
* word2vec မော်ဒယ်ကို ကြိုတင်လေ့ကျင့်ထားတဲ့ ဝေါဟာရနဲ့ dataset ကို load လုပ်ပါ။ Dataset ကို load လုပ်တဲ့အခါ အသုံးပြုမယ့် ဝေါဟာရကို သတ်မှတ်နိုင်ပါတယ်။

နောက်ဆုံးနည်းလမ်းက ပိုမိုလွယ်ကူသလို PyTorch `torchtext` framework မှာ embedding တွေအတွက် built-in support ပါဝင်တာကြောင့် ပိုမိုအဆင်ပြေပါတယ်။  
ဥပမာအားဖြင့် GloVe-based vocabulary ကို အောက်ပါနည်းလမ်းဖြင့် instantiate လုပ်နိုင်ပါတယ်-  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


အသုံးပြုနိုင်သော ဝေါဟာရတွင် အခြေခံ လုပ်ဆောင်ချက်များ ရှိပါသည်။
* `vocab.stoi` dictionary သည် စကားလုံးကို ၎င်း၏ အညွှန်းနံပါတ်သို့ ပြောင်းလဲရန် အကူအညီပေးသည်။
* `vocab.itos` သည် ဆန့်ကျင်ဘက်လုပ်ဆောင်ချက်ကို ပြုလုပ်သည် - နံပါတ်ကို စကားလုံးသို့ ပြောင်းလဲသည်။
* `vocab.vectors` သည် embedding vectors များပါဝင်သော array ဖြစ်ပြီး၊ စကားလုံး `s` ၏ embedding ကို ရယူရန် `vocab.vectors[vocab.stoi[s]]` ကို အသုံးပြုရမည်။

ဤသည်မှာ **kind-man+woman = queen** ဆိုသော သင်္ချာကို ပြသရန် embedding များကို ပြောင်းလဲအသုံးပြုသည့် ဥပမာတစ်ခုဖြစ်သည် (အလုပ်ဖြစ်စေရန် coefficient ကို အနည်းငယ် ပြင်ဆင်ခဲ့ရသည်)။


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

ဒေါင်းလုပ်များကို အသုံးပြု၍ classifier ကို လေ့ကျင့်ရန်အတွက်၊ ကျွန်ုပ်တို့၏ dataset ကို GloVe vocabulary ကို အသုံးပြု၍ encode လုပ်ရန်အရင်လိုအပ်သည်။


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

အပေါ်တွင်မြင်တွေ့ခဲ့သည့်အတိုင်း၊ vector embedding အားလုံးကို `vocab.vectors` matrix တွင်သိမ်းဆည်းထားသည်။ ၎င်းကို embedding layer ၏ weight များထဲသို့ရိုးရှင်းသောကူးယူမှုကိုအသုံးပြု၍ weight များထည့်သွင်းရန်အလွန်လွယ်ကူစေသည်။


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

ကျွန်ုပ်တို့ အချက်အလက်များတွင် ပါဝင်သော စကားလုံးများအနက် အချို့သည် ကြိုတင်လေ့ကျင့်ထားသော GloVe ဝေါဟာရတွင် မပါဝင်သောကြောင့် တိကျမှုမှာ အရေးပါသော တိုးတက်မှု မရှိခြင်း၏ အကြောင်းအရင်းများထဲမှ တစ်ခုဖြစ်သည်။ ထို့ကြောင့် အဆိုပါ စကားလုံးများကို အဓိကအားဖြင့် မထည့်သွင်းထားပါ။ ဤအချက်ကို ကျော်လွှားရန် ကျွန်ုပ်တို့၏ အချက်အလက်များပေါ်တွင် ကိုယ်ပိုင် embedding များကို လေ့ကျင့်နိုင်ပါသည်။


## အကြောင်းအရာအခြေခံ Embeddings

Word2Vec ကဲ့သို့သော ရိုးရာ pretrained embedding ကိုယ်စားပြုမှုများ၏ အဓိက ကန့်သတ်ချက်တစ်ခုမှာ စကားလုံးအဓိပ္ပါယ်ရှင်းလင်းမှု၏ ပြဿနာဖြစ်သည်။ Pretrained embeddings များသည် စကားလုံးများ၏ အချို့သော အဓိပ္ပါယ်များကို အကြောင်းအရာအတွင်းမှ ဖမ်းဆီးနိုင်သော်လည်း၊ စကားလုံးတစ်လုံး၏ အားလုံးဖြစ်နိုင်သော အဓိပ္ပါယ်များကို တစ်ခုတည်းသော embedding အတွင်းတွင် encode လုပ်ထားသည်။ ဒါက Downstream မော်ဒယ်များတွင် ပြဿနာများ ဖြစ်ပေါ်စေတတ်သည်၊ အကြောင်းမူကား 'play' ကဲ့သို့သော စကားလုံးများသည် အသုံးပြုသည့် အကြောင်းအရာပေါ်မူတည်၍ အဓိပ္ပါယ်များကွဲပြားနိုင်သည်။

ဥပမာအားဖြင့် 'play' စကားလုံးသည် အောက်ပါ စာကြောင်းနှစ်ခုတွင် အဓိပ္ပါယ်ကွဲပြားမှုများ ရှိသည်။
- ကျွန်တော် **play** တစ်ခုကို ရုံတော်မှာ သွားကြည့်ခဲ့တယ်။
- John က သူ့သူငယ်ချင်းတွေနဲ့ **play** လုပ်ချင်တယ်။

အထက်ပါ pretrained embeddings များသည် 'play' စကားလုံး၏ အဓိပ္ပါယ်နှစ်ခုလုံးကို တစ်ခုတည်းသော embedding အတွင်းတွင် ကိုယ်စားပြုထားသည်။ ဒီကန့်သတ်ချက်ကို ကျော်လွှားရန် **ဘာသာစကားမော်ဒယ်** အပေါ် အခြေခံထားသော embeddings များကို တည်ဆောက်ရန် လိုအပ်သည်။ ဘာသာစကားမော်ဒယ်သည် စာသားများ၏ အကြီးမားသော corpus အပေါ်တွင် လေ့ကျင့်ထားပြီး စကားလုံးများကို အကြောင်းအရာကွဲပြားမှုများအတွင်းတွင် ဘယ်လိုပေါင်းစည်းနိုင်သည်ကို *သိ* နိုင်သည်။ Contextual embeddings ကို ဆွေးနွေးခြင်းသည် ဒီသင်ခန်းစာ၏ အကျိုးကျေးဇူးအတွင်း မပါဝင်သော်လည်း၊ ဘာသာစကားမော်ဒယ်များကို နောက်ထပ် unit တွင် ပြန်လည်ဆွေးနွေးမည်ဖြစ်သည်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူရင်းဘာသာစကားဖြင့် အာဏာတရ အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
